# Learning range test
Apply a learning range test as proposed in [Cyclical Learning Rates for Training Neural Networks](https://arxiv.org/pdf/1506.01186.pdf). <br>

Summary of the test: <br>

1. Run your model for several epochs while letting the learning rate increase linearly between low and high LR values.
2. Next, plot the accuracy versus learning rate. Note the learning rate value when the accuracy starts to increase and 
when the accuracy slows, becomes ragged, or starts to fall. These two learning rates are good choices for bounds; 

3. Set `base_lr` to the first value and set `max_lr` to the latter value. Alternatively, one can use the rule 
of thumb that the optimum learning rate is usually within a factor of two of the largest one that converges and set 
`base_lr` to 1/3 or 1/4 of `max_lr`.

In [1]:
import numpy as np
import torch 
from transformers import (
    TrainingArguments,
    Trainer
)
from utils import retrieve_model, retrieve_parameters

if torch.cuda.is_available():
  print("Good to go!")
else:
  print("Please connect to a GPU to run this notebook.")

/home/admin/miniconda3/envs/oml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Please connect to a GPU to run this notebook.


/home/admin/miniconda3/envs/oml/lib/python3.12/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


#### Free parameters

In [2]:
task_name = "qa"        # choose amongst "sa", "nli", "ner", "qa"
low_lr = 0.01
high_lr = 1

#### Fixed parameters

In [3]:
# model
model_name = "roberta-base"

# training loop parameters
params = retrieve_parameters("parameters.yml")
ta_kwargs = params["training_loop"]
ta_kwargs["output_dir"] = "models/{}_finetuned-{}_lrrange".format(model_name, task_name)
ta_kwargs["logging_dir"] = "./tensorboard/{}_finetuned-{}_lrrange".format(model_name, task_name)
ta_kwargs["num_train_epochs"] = 5
ta_kwargs["evaluation_strategy"] = "steps"
ta_kwargs["eval_steps"] = 100


In [4]:
# task
task = retrieve_model(task_name, model_name)

# optimizer
optimizer = torch.optim.SGD(task.model.parameters(), lr=high_lr)

# scheduler
iter_per_epoch = task.dataset["train"].num_rows/ta_kwargs["per_device_train_batch_size"]
n_iters = iter_per_epoch*ta_kwargs["num_train_epochs"]
start_factor = low_lr/high_lr
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=start_factor, total_iters=n_iters)

# training
training_args = TrainingArguments(**ta_kwargs)

trainer = Trainer(
    task.model,
    args=training_args,
    optimizers=(optimizer, scheduler),
    train_dataset=task.encoded_dataset["train"],
    eval_dataset=task.encoded_dataset["validation"],
    tokenizer=task.tokenizer,
    data_collator=task.data_collator,
    compute_metrics=task.compute_metrics
)

trainer.train()

Map: 100%|██████████| 10570/10570 [00:03<00:00, 3458.82 examples/s]
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 10570/10570 [00:03<00:00, 2836.28 examples/s]
/home/admin/miniconda3/envs/oml/lib/python3.12/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/admin/miniconda3/envs/oml/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfigurati

{'loss': 4.5159, 'grad_norm': 44.22573471069336, 'learning_rate': 0.014520599550223137, 'epoch': 0.02}


Post-processing 10570 example predictions split into 10790 features.








































































































































































































































100%|██████████| 10570/10570 [00:24<00:00, 435.68it/s]
                                                      
  0%|          | 100/22145 [52:01<77:29:12, 12.65s/it]

{'eval_loss': 3.144231081008911, 'eval_exact_match': 21.95837275307474, 'eval_f1': 36.27605615087538, 'eval_runtime': 1944.9007, 'eval_samples_per_second': 5.548, 'eval_steps_per_second': 0.278, 'epoch': 0.02}


...

Post-processing 10570 example predictions split into 10790 features.
























































































































































































































100%|██████████| 10570/10570 [00:22<00:00, 468.81it/s]
                                                          
 39%|███▉      | 8600/22145 [71:31:15<42:00:44, 11.17s/it]

{'eval_loss': 1.1255532503128052, 'eval_exact_match': 78.1929990539262, 'eval_f1': 86.10923376099284, 'eval_runtime': 1897.2703, 'eval_samples_per_second': 5.687, 'eval_steps_per_second': 0.285, 'epoch': 1.94}


 39%|███▉      | 8700/22145 [71:49:48<41:40:10, 11.16s/it]   

{'loss': 1.2063, 'grad_norm': 20.737070083618164, 'learning_rate': 0.40329216086943026, 'epoch': 1.96}
